# Example usage
Here is a demonstration of using "organage" to estimate organ-specific biological age from SomaScan data. 

"organage" requires SomaScan v4 (5k proteins) or v4.1 (7k proteins) data. 

In [ ]:
from organage import OrganAge
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy.interpolate import interp1d
from scipy import stats
import numpy as np


: 

# Load data

### sample x metadata dataframe
- index should be sample name
- "Age" and "Sex_F" are required columns in this dataframe.
- other columns are optional

In [ ]:
md_hot = pd.read_csv("../tests/md_hot.csv")
md_hot = md_hot.set_index("ID")
md_hot

: 

### sample x protein expression dataframe
- index should be sample name
- columns should be "SeqId"s from SomaScan data. 
- values should be ANML normalized expression data in RFU units. These values are the raw values from the '.anmlSMP.adat' file provided by Somalogic


In [ ]:
df_prot = pd.read_csv("../tests/df_prot.csv")
df_prot = df_prot.set_index("ID")
df_prot

: 

# Calculate organ age gaps


In [ ]:
data = OrganAge.CreateOrganAgeObject()
data.add_data(md_hot, df_prot)
data.normalize(assay_version="v4.1")  #requires "v4" 5k, or "v4.1" 7k
res = data.estimate_organ_ages()
res

: 

In [ ]:
toplot = res.loc[res.Organ=="Heart"]
toplot = toplot.sort_values("Age")
sns.scatterplot(data=toplot, x="Age", y="Predicted_Age", 
                hue="AgeGap_zscored", palette='coolwarm', hue_norm=(-3,3))                
plt.plot(toplot.Age, toplot.yhat_lowess)
plt.show()


: 

# Recalculate age gap based on cohort age gap distribution
The default method for this package calculates age gaps based on the distribution of predicted ages from the models' training cohort. 


It may be necessary to recalculate age gaps based on the distribution of the desired cohort if there are strong cohort effects. We show how to do this below.
1. for each organ...
2. derive lowess curve for predicted versus chronological age within a single cohort
3. calculate age gap as the predicted age - lowess curve
4. z-score

In [ ]:
todf=[]
FRAC=2/3

#for each organ
for organ in set(res.Organ):
    res_sub = res.loc[res.Organ==organ].copy()

    #lowess curve
    lowess = sm.nonparametric.lowess
    lowess_fit=lowess(res_sub.Predicted_Age.to_numpy(), res_sub.Age.to_numpy(), frac=FRAC, it=5)
    lowess_fit_int = interp1d(lowess_fit[:,0], lowess_fit[:,1], bounds_error=False, kind='linear', fill_value='extrapolate') 
    y_lowess=lowess_fit_int(res_sub.Age)
    res_sub["yhat_lowess_cohort"] = y_lowess

    #age gap
    res_sub["AgeGap_cohort"] = res_sub["Predicted_Age"] - res_sub["yhat_lowess_cohort"]

    #z-score
    res_sub["AgeGap_cohort_zscored"] = stats.zscore(res_sub["AgeGap_cohort"])
    todf.append(res_sub)

res_all = pd.concat(todf)
res_all


: 

In [ ]:
toplot = res_all.loc[res_all.Organ=="Heart"]
toplot = toplot.sort_values("Age")
sns.scatterplot(data=toplot, x="Age", y="Predicted_Age", 
                hue="AgeGap_cohort_zscored", palette='coolwarm', hue_norm=(-3,3))                
plt.plot(toplot.Age, toplot.yhat_lowess, label="training cohort lowess")
plt.plot(toplot.Age, toplot.yhat_lowess_cohort, label="this cohort lowess")
plt.legend()
plt.show()


: 

lowess regression may be unnecessary if the distribution is linear, and/or lowess may be overfit with low sample size. 

In [ ]:
#ie. low sample size causes Artery lowess to overfit
toplot = res_all.loc[res_all.Organ=="Artery"]
toplot = toplot.sort_values("Age")
sns.scatterplot(data=toplot, x="Age", y="Predicted_Age", 
                hue="AgeGap_cohort_zscored", palette='coolwarm', hue_norm=(-3,3))                
plt.plot(toplot.Age, toplot.yhat_lowess, label="training cohort lowess")
plt.plot(toplot.Age, toplot.yhat_lowess_cohort, label="this cohort lowess")
plt.legend()
plt.show()


: 

can either increase lowess paramter "frac" to 1 or use linear regression.

In [ ]:
#linear regression of predicted versus chronological age
todf=[]

#for each organ
for organ in set(res.Organ):
    res_sub = res.loc[res.Organ==organ].copy()

    #lowess curve
    ols = sm.OLS(res_sub.Predicted_Age, sm.add_constant(res_sub[["Age"]])).fit()    
    res_sub["yhat_linear_cohort"] = ols.predict(sm.add_constant(res_sub[["Age"]]))

    #age gap
    res_sub["AgeGap_cohort"] = ols.resid

    #z-score
    res_sub["AgeGap_cohort_zscored"] = stats.zscore(res_sub["AgeGap_cohort"])
    todf.append(res_sub)

res_all = pd.concat(todf)
res_all


: 

In [ ]:
toplot = res_all.loc[res_all.Organ=="Artery"]
toplot = toplot.sort_values("Age")
sns.scatterplot(data=toplot, x="Age", y="Predicted_Age", 
                hue="AgeGap_cohort_zscored", palette='coolwarm', hue_norm=(-3,3))
plt.plot(toplot.Age, toplot.yhat_lowess, label="training cohort lowess")
plt.plot(toplot.Age, toplot.yhat_linear_cohort, label="this cohort linear")
plt.legend()
plt.show()


: 

: 